In [1]:
# Imports
import os
import json
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
verbose = False #Set to true to see more output information
DATA_DIR = "/data/extracted"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

# count tokens
def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

#lookup functions
rag_lib="" 

def get_library_info(lib_name: str)->str:  
    if lib_name=="CAA File":    
        return ""
    else:
        return rag_lib
def get_function_info(lib_name: str, fn_name:str)->str: 
    if fn_name=="FB_SendEMail":  
        with open(f'../{DATA_DIR}/{fn_name}.txt', 'r') as file:
            response=file.read()
            return response   
    elif fn_name=="FB_Pop3EMailClient":
        with open(f'../{DATA_DIR}/{fn_name}.txt', 'r') as file:
            response=file.read()
            return response 
    else:
        return ""
    
#Just a test to check if the function is called when the function is marked as not found INTO PROVIDE 
def get_not_found_info(fn_name:str)->str: 
    return "NOT FOUND"    
    
    
available_functions = {
                "get_library_info": get_library_info,
                "get_function_info": get_function_info,
                "get_not_found_info": get_not_found_info                
        }

#GPT prompt functions
functions = [
   {
      "name":"get_library_info",
      "description":"Get the information about the functions included in a library.",
      "parameters": {
         "type":"object",
         "properties":{
            "lib_name":{
               "type":"string",               
               "description": "The name of the library to get information about. (examples: FileFormatUtility,HttpHandling,TimeSync)"
            }       
         },
         "required":["lib_name"]
      },
   }, 
   {
    "name":"get_function_info",
    "description":"Get detailed information about the functions included in a specific library.",
    "parameters": {
        "type":"object",
        "properties":{
        "lib_name":{
            "type":"string",               
            "description": "The name of the library to get information about. (examples: FB_SntpClient,HttpHandling,TimeSync)"
        },
        "fn_name":{
            "type":"string",               
            "description": "The name of the function block from the library I need to(examples: FB_Pop3EMailClient,FBSendMail)"
        },         
        },
        "required":["lib_name","fn_name"]
        },
    },
     {
    "name":"get_not_found_info",
    "description":"Get detailed information about the functions described as NOT FOUND.",
    "parameters": {
        "type":"object",
        "properties":{
         "fn_name":{
            "type":"string",               
            "description": "The name of the function described with [NOT FOUND]"
        },         
        },
        "required":["fn_name"]
        },
    },    
]

def load_info(response_message):
    function_name = response_message.function_call.name
    if function_name:   
        function_to_call = available_functions[function_name] 
        function_args = json.loads(response_message.function_call.arguments)
        print(f"\nModel wants to call a function '{function_name}' with arguments: {function_args}") 
                
        function_response = function_to_call(**function_args)
        #print(f"Function '{function_name}' returned: {function_response}")
        
        total_tokens=count_tokens(function_response)
        print (f'Total function tokens: {total_tokens}')
        
        # Append function info to messages
        messages.append( # adding assistant response to messages
            {
                "role": response_message.role,
                "function_call": {
                    "name": function_name,
                    "arguments": response_message.function_call.arguments,
                },
                "content": None
            }
        )
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": function_name,
                "content":function_response,
            }
       ) 

def generate_code():    
    openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        functions=functions,
        function_call="auto"
    )
    return openai_response.choices[0].message

In [2]:
#This command should invoke the function get_not_found_info because of the TLS 1.2 setting
user_command = f'''
Generate a small program in IEC61131-3 that receives an email using following parameters:
1. from: "bart_simpson@foo.com"
2. if received email contains "joke" send an email with following parameters:
    1. To: "alert@foo.com"
    2. Subject: "Alert email"
    3. Body: "A joke email has been received"
    4. From: "sender@se.com"
    5. Authentication required.
    6. Use Protocol TLS 1.2
    7. Message should be sent with low priority.
    8. No Authentication required.
    9. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.
    10. Consider all the case of failure, for example if the email server is not reachable.
3. else saves the content of the email to a file named "print_email.txt".
# '''

In [3]:
#This command should NOT invoke the function get_not_found_info since all the data types are
# user_command = f'''
# Generate a small program in IEC61131-3 that receives an email using following parameters:
# 1. from: "bart_simpson@foo.com"
# 2. if received email contains "joke" send an email with following parameters:
#     1. To: "alert@foo.com"
#     2. Subject: "Alert email"
#     3. Body: "A joke email has been received"
#     4. From: "sender@se.com"
#     5. Authentication required.
#     6. Use Protocol TLS 1.2
#     7. Message should be sent with low priority.
#     8. No Authentication required.
#     9. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.
#     10. Consider all the case of failure, for example if the email server is not reachable.
# 3. else saves the content of the email to a file named "print_email.txt".
# # '''

In [4]:
library_rules = '''
These are the rules to follow to retrieve the information about an external library:

1. if name of the library is with "CAA File" don't invoke 'get_library_info' nor 'get_function_info'.
2. 'get_library_info' return information about the functions included in a library, it must to be the first function to call.
3. 'get_function_info' must be called to retrieve all the information about a specific function.
4. if a function is described with [NOT FOUND], the function 'get_not_found_info' must be called.

example:
1. You need to use one or more function contained into 'Lib001' library.
2. You need to call get_library_info with lib_name = 'Lib001'.
3. The function return info about the library stating it contains 2 functions: 'FB_RxData' and 'FB_TxData'.
4. if you need to use 'FB_RxData' you need to call get_function_info with lib_name = 'Lib0001 and fn_name = 'FB_RxData'.
5. if you need to use 'FB_TxData' you need to call get_function_info with lib_name = 'Lib001' and fn_name = 'FB_TxData'.
'''

In [5]:
rag_lib='''
1. `FB_SendEMail`: This function block includes the related functions for sending emails. Each instance handles one SMTP connection. It allows you to send emails. After the function block has been enabled and is being executed, a TCP connection to the email server is established using the user credentials that have been submitted using iq_stCredentials.

2. `FB_Pop3EMailClient`: This function block includes the related functions for receiving and deleting emails using POP3. Each instance handles one POP3 connection. It allows you to receive and delete emails. By using attachments of received emails you are able to get input for several system features which are based on files located on the system memory.

3. `FC_EtResultToString`: This function is used to convert an enumeration element of type ET_Result to a variable of type STRING. It provides a way to convert the enumeration type to a string for easier interpretation and display.
'''

In [6]:
libraries='''
Library title,Company,Category,Component,Description
CollisionDetection,Schneider Electric,Application,PacDrive Robotics,This library offers a set of functions to perform a collision check between two or more objects and a distance calculation between two or more objects. For more information, refer to the CollisionDetection Library Guide.
EMailHandling,Schneider Electric,Communication,Internet Protocol Suite, This library supports the implementation of an email client and provides function blocks for sending SMTP (Simple Mail Transfer Protocol) and receiving POP3 (Post Office Protocol) emails.NOTE: The communication is implemented using the TcpUdpCommunication library.AlarmManager,intern,Intern > AlarmManager,Core Repository,This library contains the components for the alarm handling.
CAA File,CAA Technical Workgroup,System,Core Repository,This library provides function blocks for accessing file directory systems and files.

'''
library_system_prompt=f'''
You are an assistant able to select a library based on input requirements.
Your goal is to return the library names selecting from the one provided in the context.
Favor the use of external libraries over internal libraries.
Return only the name of the libraries that matches the user input separated by a comma, if name of the library starts with CAA then append "from CodeSys" to the name of the library.

CONTEXT:
```
{libraries}
```
'''

In [7]:

# Detect libraries to use
messages=[]
messages.append({'role': 'system', 'content': library_system_prompt})
messages.append({'role': 'user', 'content': user_command})   

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

target_libraries= openai_response.choices[0].message.content
print(target_libraries)

EMailHandling, CAA File from CodeSys


In [8]:
# system message
system_message = f'''
You are an assistant with knowledge of the following topics:
1. IEC61131-3 languages
2. Structured Text
3. Function Block Diagram
4. IEC61131-3 coding standards
5. IEC61131-3 best practices
6. IEC61131-3 coding guidelines
7. IEC61131-3 programming
8. IEC61131-3 programming languages
9. Schneider Electric EcoStruxure Control Expert
10. Schneider Electric EcoStruxure Machine Expert
11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

Your job is to generate small examples of code using exclusively IEC61131-3 Structured Text base on user input.
You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

The code should make use of the following {target_libraries} libraries.

In order to understand how to use the libraries, you can use the following indications:

```
{library_rules}
```

'''

In [9]:
messages=[]
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': user_command})   

sys_token=count_tokens(system_message)
cmd_tokens=count_tokens(user_command)

print(f"system_message tokens: {sys_token}")
print(f"user_message tokens: {cmd_tokens}")

isCodeGenerated=False

while(isCodeGenerated==False): 
    print("Generating code...")    
    response = generate_code()    
    if (response.function_call):
        load_info(response)
    else:
        isCodeGenerated=True
        print("Code generation completed!")        
        print(response.content)

system_message tokens: 482
user_message tokens: 211
Generating code...

Model wants to call a function 'get_library_info' with arguments: {'lib_name': 'EMailHandling'}
Total function tokens: 186
Generating code...

Model wants to call a function 'get_library_info' with arguments: {'lib_name': 'CAA File'}
Total function tokens: 0
Generating code...

Model wants to call a function 'get_function_info' with arguments: {'lib_name': 'EMailHandling', 'fn_name': 'FB_SendEMail'}
Total function tokens: 3635
Generating code...

Model wants to call a function 'get_not_found_info' with arguments: {'fn_name': 'ET_TLSPROTOCOL'}
Total function tokens: 2
Generating code...

Model wants to call a function 'get_not_found_info' with arguments: {'fn_name': 'ET_CERTVERIFYMODE'}
Total function tokens: 2
Generating code...

Model wants to call a function 'get_function_info' with arguments: {'lib_name': 'EMailHandling', 'fn_name': 'FB_Pop3EMailClient'}
Total function tokens: 3898
Generating code...

Model want